<h1>Haircut Suggestions using Stable Diffusion and face swapping</h1>

<h3>Import modules for image generation</h3>

In [49]:
import webuiapi
import PIL

<h3>Connect to your automatic1111</h3>
<p>You should have it already running with --api argument</p>

In [50]:
api = webuiapi.WebUIApi(host='127.0.0.1', port=7861)

<h3>Read portrait image</h3>

In [51]:
input_img = PIL.Image.open('your_test_pportrait_here.jpg')

<h3>Create ControlNet Unit</h3>

In [52]:
unit1 = webuiapi.ControlNetUnit(input_image=input_img, module='openpose', model='control_v11p_sd15_openpose [cab727d4]', weight=0.6)

<h3>Generate new image</h3>

In [ ]:
r = api.txt2img(
    prompt = 'photo of a woman, (Faux Hawk Haircut:1.2), wearing a white t-shirt, neutral light gray background, detailed (wrinkles, blemishes, folds, moles, veins, pores, skin imperfections:1.1), highly detailed glossy eyes, (looking at the camera), specular lighting, dslr, ultra quality, sharp focus, tack sharp, dof, film grain, centered, Fujifilm XT3, crystal clear',
    negative_prompt = 'hat, headscarf, naked, nude, out of frame, tattoo, b&w, sepia, (blurry un-sharp fuzzy un-detailed skin:1.4), (twins:1.4), (geminis:1.4), (wrong eyeballs:1.1), (cloned face:1.1), (perfect skin:1.2), (mutated hands and fingers:1.3), disconnected hands, disconnected limbs, amputation, (semi-realistic, cgi, 3d, render, sketch, cartoon, drawing, anime, doll, overexposed, photoshop, oversaturated:1.4)',
    controlnet_units=[unit1],
    cfg_scale = 3.5,
    steps = 18,
    sampler_name = "DPM++ 2M Karras",
    restore_faces = True,
    override_settings = {
        "sd_model_checkpoint": "realisticVisionV60B1_v51VAE.safetensors [ef76aa2332]",
    },
)
new_image = r.images[0]

<h3>Import modules for face swapping</h3>

In [56]:
import insightface
from insightface.app import FaceAnalysis
from insightface.data import get_image as ins_get_image

import numpy as np

<h3>Initiate face capturing model</h3>
<p>There will be some warnings here, ignore them</p>

In [ ]:
app = FaceAnalysis(name='buffalo_l')
app.prepare(ctx_id=0, det_size=(640, 640))

<h3>Prepare images for face swapping</h3>

In [61]:
input_image_array = np.array(input_img)
new_image_array = np.array(new_image)

<h3>Get faces from images</h3>

In [63]:
input_image_face = app.get(input_image_array)[0]
new_image_face = app.get(new_image_array)[0]

/home/alex/Haircut_SD/.venv/lib/python3.10/site-packages/insightface/utils/transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4


<h3>Initiate swapper model</h3>

In [64]:
swapper = insightface.model_zoo.get_model('inswapper_128.onnx',
                                          download=False,
                                          download_zip=False)

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
inswapper-shape: [1, 3, 128, 128]


<h3>Swap faces</h3>

In [65]:
result = new_image_array.copy()
result = swapper.get(result, new_image_face, input_image_face, paste_back=True)

<h3>Show result</h3>

In [66]:
PIL.Image.fromarray(np.uint8(result)).convert('RGB')